In [3]:
import json_lines, gzip, csv

In [8]:
with gzip.open('../../dmoz/content_url_topics.csv.gz', 'rt') as f:
    topics_by_url = {url: topics for i, (url, topics) in enumerate(csv.reader(f))
                     if i != 0}

In [9]:
len(topics_by_url)

3222503

In [12]:
from collections import defaultdict
urls_by_topic = defaultdict(list)
for url, topics in topics_by_url.items():
    for topic in topics.split('/'):
        urls_by_topic[topic].append(url)

In [13]:
topic_counts = sorted(((topic, len(urls)) for topic, urls in urls_by_topic.items()), key=lambda x: x[1], reverse=True)

In [15]:
topic_counts[:100]

[('Top', 3222100),
 ('World', 1530547),
 ('Regional', 1115260),
 ('North_America', 579081),
 ('United_States', 552733),
 ('Localities', 432721),
 ('Deutsch', 414555),
 ('Europa', 376914),
 ('Europe', 310769),
 ('Business_and_Economy', 292272),
 ('Deutschland', 194285),
 ('Français', 192438),
 ('United_Kingdom', 176963),
 ('Arts', 164999),
 ('Society', 162427),
 ('Japanese', 155173),
 ('Städte_und_Gemeinden', 144989),
 ('Business', 138975),
 ('Italiano', 134222),
 ('Régional', 113500),
 ('England', 110666),
 ('Español', 108070),
 ('Russian', 105573),
 ('Wirtschaft', 97638),
 ('Sports', 94968),
 ('Landkreise', 93672),
 ('Shopping', 86152),
 ('Regionale', 82628),
 ('Travel_and_Tourism', 82023),
 ('France', 80998),
 ('Education', 80778),
 ('Italia', 79271),
 ('Canada', 77908),
 ('Religion_and_Spirituality', 76906),
 ('Science', 76351),
 ('Health', 75002),
 ('Località', 73578),
 ('Computers', 73534),
 ('Nederlands', 72664),
 ('Society_and_Culture', 70928),
 ('S', 65966),
 ('Christianity', 6

In [17]:
printed = 0
for url, topics in topics_by_url.items():
    if '/A/' in topics:
        print(topics, url)
        printed += 1
        if printed > 5:
            break

Top/Health/Pharmacy/Drugs_and_Medications/A/Anakinra https://medlineplus.gov/druginfo/meds/a602001.html
Top/Regional/North_America/United_States/Georgia/Localities/A/Athens/Travel_and_Tourism/Lodging http://hamptoninn3.hilton.com/en/hotels/georgia/hampton-inn-athens-AHNGAHX/index.html
Top/Games/Video_Games/Roleplaying/A/Age_of_Conan_-_Hyborian_Adventures http://pc.gamespy.com/pc/age-of-conan-hyborian-adventures/
Top/World/Deutsch/Regional/Europa/Deutschland/Nordrhein-Westfalen/Städte_und_Gemeinden/A/Aachen/Gesellschaft/Politik http://www.spd-aachen.de/
Top/Regional/North_America/United_States/Alabama/Localities/A/Attalla/Education http://ies.ecboe.org/
Top/World/Deutsch/Regional/Europa/Deutschland/Nordrhein-Westfalen/Städte_und_Gemeinden/A/Aachen/Sport/Tanzen http://www.dancing-and-more.de/


In [41]:
import fasttext
from hh_page_clf.pretraining.dmoz_fasttext import to_single_line

In [19]:
m = fasttext.load_model('../dmoz-ng1-mc10-mcl10.model.bin.bin')

In [22]:
import json
with gzip.open('../datasets/iarpa-blogs.json.gz', 'rt') as f:
    pages = json.load(f)

In [42]:
import html_text
texts = [to_single_line(html_text.extract_text(page['html'])) for page in pages['pages']]

In [43]:
from pprint import pprint
for item, text in zip(pages['pages'], texts[:10]):
    pprint((item['relevant'], m.predict_proba([text], k=10)))

(False,
 [[('__label__Science', 0.0332031),
   ('__label__Computers', 0.0292969),
   ('__label__Regional', 0.0214844),
   ('__label__Society', 0.0214844),
   ('__label__Europe', 0.0175781),
   ('__label__Arts', 0.015625),
   ('__label__Health', 0.0136719),
   ('__label__Internet', 0.0117188),
   ('__label__United_Kingdom', 0.0117188),
   ('__label__Software', 0.00976564)]])
(True,
 [[('__label__Computers', 0.0351563),
   ('__label__Society', 0.0351563),
   ('__label__Arts', 0.0292969),
   ('__label__Games', 0.0195313),
   ('__label__Internet', 0.0175781),
   ('__label__RFCs', 0.015625),
   ('__label__Video_Games', 0.0136719),
   ('__label__Titles', 0.0136719),
   ('__label__Issues', 0.0136719),
   ('__label__Movies', 0.0117188)]])
(True,
 [[('__label__Regional', 0.046875),
   ('__label__Society', 0.0429688),
   ('__label__Science', 0.0292969),
   ('__label__Computers', 0.0214844),
   ('__label__North_America', 0.0214844),
   ('__label__United_States', 0.0195313),
   ('__label__Europe',

In [44]:
pages['pages'][0]['url']

'http://2014.hack.lu/index.php/list'